# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [ ]:
import os
import glob
import psycopg2
import pandas as pd
from pandas.io.json import json_normalize
from sql_queries import *

- Discuss the purpose of this database in the context of the startup, Sparkify, and their analytical goals.
- State and justify your database schema design and ETL pipeline.
- [Optional] Provide example queries and results for song play analysis.


# Sparkify -- Data Modeling with Postgres

The goal of this database is to create an analytical data framework to identify users' behaviors. Currently all data is stored in logs and is not easily accessible to our analytics team. Our goal is to process these JSON files that house the raw Sparkify data into a consumable format for downstream teams.

Our team has created an ETL pipeline to parse the log information into a star schema made (fact and dimension tables) that makes analyzing events simpler. The fact table, `songplays`, lets consumers get all the relevant information needed for simple aggregations, like `COUNT`s of users or agents. If one neededs to deep dive a particular user, artist, or song, they can join the appropriate table back to the fact table to get their information. For example, if we needed so `COUNT` the number of unique songs listened to by user name, we could do something like the following:

~~~~
SELECT
    u.first_name | u.last_name AS user_name
    ,COUNT(s.song_id)
FROM
    songplays s
    JOIN users u
        ON s.user_id = u.user_id
GROUP BY
    1
~~~~

# Data Structure

Fact Table
    
    songplays - records in log data associated with song plays i.e. records with page NextSong
        songplay_id
        , start_time
        , user_id
        , level
        , song_id
        , artist_id
        , session_id
        , location
        , user_agent

Dimension Tables

    users - users in the app
        user_id
        , first_name
        , last_name
        , gender
        , level
    
    songs - songs in music database
        song_id
        , title
        , artist_id
        , year
        , duration
    
    artists - artists in music database
        artist_id
        , name
        , location
        , latitude
        , longitude
    
    time - timestamps of records in songplays broken down into specific units
        start_time, hour, day, week, month, year, weekday


In [ ]:
import create_tables
create_tables.create_database()

In [ ]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [ ]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [ ]:
song_filepath = 'data/song_data'

In [ ]:
song_files = get_files(filepath=song_filepath)

In [ ]:
song_files[0]

In [ ]:
df = [pd.DataFrame.from_dict(pd.read_json(i, typ='series')).T for i in song_files]
df[0]

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [ ]:
song_data = list(pd.concat(df)[['title', 'song_id', 'artist_id', 'year', 'duration']].iloc[0,:])
song_data

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [ ]:
cur.execute(song_table_create)
conn.commit()

cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [ ]:
artist_data = list(pd.concat(df)[['artist_id','artist_name','artist_latitude', 'artist_longitude']].iloc[0,:])
artist_data

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [ ]:
cur.execute(artist_table_create)
conn.commit()

cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [ ]:
log_filepath = 'data/log_data'

In [ ]:
log_files = get_files(filepath=log_filepath)
log_files[0]

In [ ]:
log_df = pd.concat([pd.DataFrame.from_dict(pd.read_json(i, lines=True)) for i in log_files])
log_df.head()

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [ ]:
log_df = log_df[log_df['page'] == 'NextSong']
log_df.head()

In [ ]:
log_df['ts'] = pd.to_datetime(log_df['ts'], unit='ms')
log_df.head()

In [ ]:
time_df = pd.DataFrame({
    'ts':log_df['ts']
    ,'hour':log_df['ts'].dt.hour
    ,'day':log_df['ts'].dt.day
    ,'week':log_df['ts'].dt.week
    ,'month':log_df['ts'].dt.month
    ,'year':log_df['ts'].dt.year
    ,'weekday':log_df['ts'].dt.weekday
})

In [ ]:
# time_df = log_df.join(time_data)
time_df.head()

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [ ]:
cur.execute(time_table_create)
conn.commit()

for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()
print(list(row))

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [ ]:
user_df = log_df[['userId', 'firstName', 'lastName', 'gender', 'level']]

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [ ]:
cur.execute(user_table_create)
conn.commit()

for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
cur.execute(songplay_table_create)
conn.commit()

for index, row in log_df.iterrows():
    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.artist, row.song, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.

In [1]:
from etl import main
main()

71 files found in data/song_data
1/71 files processed.
2/71 files processed.
3/71 files processed.
4/71 files processed.
5/71 files processed.
6/71 files processed.
7/71 files processed.
8/71 files processed.
9/71 files processed.
10/71 files processed.
11/71 files processed.
12/71 files processed.
13/71 files processed.
14/71 files processed.
15/71 files processed.
16/71 files processed.
17/71 files processed.
18/71 files processed.
19/71 files processed.
20/71 files processed.
21/71 files processed.
22/71 files processed.
23/71 files processed.
24/71 files processed.
25/71 files processed.
26/71 files processed.
27/71 files processed.
28/71 files processed.
29/71 files processed.
30/71 files processed.
31/71 files processed.
32/71 files processed.
33/71 files processed.
34/71 files processed.
35/71 files processed.
36/71 files processed.
37/71 files processed.
38/71 files processed.
39/71 files processed.
40/71 files processed.
41/71 files processed.
42/71 files processed.
43/71 file